# Smart case study

## import libraly

In [1]:
import gdown
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import inflection
from catboost import CatBoostRegressor, Pool
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib
import numpy as np


In [2]:
output_rents = '../data/snp_dld_2024_rents.csv'
output_trans = '../data/snp_dld_2024_transactions.csv'


## Dowload dataset from google disk

In [3]:
# URL to the file on Google Drive  
url_rents = 'https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x'
url_trans = 'https://drive.google.com/uc?id=1liNykIOnfR5KRR4MXJISCZCKJYFnXQC1'
# Download the file
gdown.download(url_rents, output_rents, quiet=False)
gdown.download(url_trans, output_trans, quiet=False)

# Load data into DataFrame
snp_dld_2024_rents = pd.read_csv(output_rents)
snp_dld_2024_transactions = pd.read_csv(output_trans)
print(snp_dld_2024_rents.shape)
print(snp_dld_2024_transactions.shape)

Downloading...
From (original): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x
From (redirected): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x&confirm=t&uuid=7b9b15ff-0d0b-4495-a6f8-507a8b5c3f17
To: /media/aleksey/A6B828A60EB3956D/Github/SB-Case-Study-Materials/data/snp_dld_2024_rents.csv
100%|██████████| 278M/278M [01:09<00:00, 3.99MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1liNykIOnfR5KRR4MXJISCZCKJYFnXQC1
To: /media/aleksey/A6B828A60EB3956D/Github/SB-Case-Study-Materials/data/snp_dld_2024_transactions.csv
100%|██████████| 70.1M/70.1M [00:12<00:00, 5.57MB/s]
/tmp/ipykernel_879848/1147897152.py:9: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  snp_dld_2024_rents = pd.read_csv(output_rents)


(681447, 43)
(162806, 46)


## Preprocess

### Rentals

In [4]:
def load_data(path):
    """
    Загрузка данных из CSV файлов.
    
    :param rentals_path: Путь к файлу с данными по аренде.
    :param sales_path: Путь к файлу с данными по продажам.
    :return: Два DataFrame с данными по аренде и продажам.
    """
    df = pd.read_csv(path)

    return df

rentals = load_data(output_rents)

/tmp/ipykernel_879848/4169113296.py:9: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [5]:
def preprocess_data_rents(df):
    """
    Data preprocessing: handling missing data, encoding categorical variables, and scaling numerical features.
    
    :param df: DataFrame with raw data.
    :return: DataFrame with preprocessed data.
    """

    # Convert date columns to datetime
    df[['registration_date', 'contract_start_date', 'contract_end_date', 'req_from', 'req_to']] = df[['registration_date', 'contract_start_date', 'contract_end_date', 'req_from', 'req_to']].apply(pd.to_datetime)
    
    # Replace 't'/'f' with True/False
    df['is_freehold'] = df['is_freehold'].replace({'t': True, 'f': False})
    
    # Calculate time deltas
    df['delta_time_registration'] = (df['registration_date'] - df['contract_start_date']).dt.days
    df['delta_time_reg_from'] = (df['registration_date'] - df['req_from']).dt.days
    df['time_reg'] = (df['req_to'] - df['req_from']).dt.days
    df['time_contract'] = (df['contract_end_date'] - df['contract_start_date']).dt.days
    df['contract_amount'] = df['contract_amount'].fillna(0)
    # Sort by version number and drop duplicates
    df.sort_values(['contract_amount', 'version_number','registration_date'], ascending=False, inplace=True)
    
    # Filter out rows with non-positive contract amounts
    df = df.query('contract_amount > 0').copy()

    df = df.drop_duplicates(subset=['ejari_contract_number', 'version_number', 'contract_start_date', 'contract_end_date'], keep='first')

    # Drop unnecessary columns
    df.drop(['is_freehold_text', 'land_property_id', 'property_id', 'property_type_ar', 
             'property_subtype_ar', 'ejari_contract_number',
             'property_usage_ar', 'property_usage_id', 'project_name_ar', 'area_ar', 'nearest_landmark_ar', 
             'nearest_metro_ar', 'nearest_mall_ar', 'master_project_ar', 'ejari_property_type_id', 
             'ejari_property_sub_type_id', 'meta_ts', 'area_id', 'version_text','annual_amount',
             'contract_end_date', 'contract_start_date', 'registration_date', 'req_from', 'req_to'
            ], axis=1, inplace=True)
    
    # Identify numerical and categorical features
    numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

    # Replace NaN values in categorical features with 'missing'
    df = df.apply(lambda x: x.fillna('missing') if x.dtype == 'object' else x)
        
    categorical_features = df.select_dtypes(include=['object']).columns

    # Convert categorical features t onumerical codes
    for col in categorical_features:
        df[col] = df[col].astype('category').cat.codes

    return df, numerical_features, categorical_features


In [6]:
df_preprocess_data_rents, numerical_features_rents, categorical_features_rents = preprocess_data_rents(rentals)
df_preprocess_data_rents

,version_number,contract_amount,is_freehold,property_size_sqm,parcel_id,property_type_en,property_subtype_en,property_usage_en,total_properties,rooms,...,area_en,nearest_landmark_en,nearest_metro_en,nearest_mall_en,master_project_en,entry_id,delta_time_registration,delta_time_reg_from,time_reg,time_contract
645663,1,4.410800e+08,True,47258.20,3920513.0,0,3,6,1,NaN,...,140,1,35,4,2,94375,1852,25,30,3287
55150,1,1.963500e+08,False,33600.00,1290104.0,0,3,1,1,NaN,...,166,5,22,1,2,94368,40,10,30,3651
475651,1,1.927818e+08,True,1854.36,NaN,2,31,1,4,NaN,...,196,2,8,1,2,94373,203,20,29,3651
630696,1,1.868167e+08,False,4337.87,3730917.0,2,43,1,1,NaN,...,6,1,51,3,2,94375,364,8,30,3652
55046,1,1.683000e+08,False,23178.00,1290104.0,0,3,9,1,NaN,...,166,5,22,1,2,94368,40,10,30,3651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536873,2,5.000000e-01,False,85.60,2450511.0,2,15,6,2,NaN,...,146,5,28,0,2,94374,26,8,29,94
671589,12,1.000000e-02,True,118.20,2514313.0,2,15,6,1,NaN,...,143,5,48,0,2,94375,295,21,30,365
587543,10,1.000000e-02,True,66.44,2514114.0,2,15,6,1,NaN,...,143,5,48,0,2,94375,280,21,30,365
588961,7,1.000000e-02,True,71.67,2514114.0,2,15,6,1,NaN,...,143,5,48,0,2,94375,182,21,30,365


### Trans

In [7]:
trans = load_data(output_trans)

In [8]:

def preprocess_data_trans(df):
    """
    Data preprocessing: handling missing data, encoding categorical variables, and scaling numerical features.
    
    :param df: DataFrame with raw data.
    :return: DataFrame with preprocessed data.
    """

    df['is_offplan'] = df['is_offplan'].replace({'t': True, 'f': False})
    df['is_freehold'] = df['is_freehold_text'].replace({'Free Hold': True, 'Non Free Hold': False})
    #df.drop_duplicates(subset=['transaction_number', 'transaction_datetime', 'req_from', 'req_to', 'amount', 
    #                        'registration_type_en'], keep='last', inplace=True)

    # Filter out rows with non-positive contract amounts
    #df = df.query('amount > 0').copy()
    df['amount'] = df['amount'].fillna(0)
    df = df.query('amount > 0').copy()

    # Calculate time deltas
    df[['req_to', 'transaction_datetime', 'req_from']] = df[['transaction_datetime', 'req_from', 'req_to']].apply(pd.to_datetime)
    df['delta_time_reg_to'] = (df['req_to'] - df['transaction_datetime']).dt.days
    df['delta_time_reg_from'] = (df['req_from'] - df['transaction_datetime']).dt.days
    df['time_reg'] = (df['req_from'] - df['req_to']).dt.days

    # Sort by version number and drop duplicates
    df.sort_values(['transaction_datetime', 'amount'], ascending=False, inplace=True)
    df.drop_duplicates(subset=['transaction_number', 'transaction_datetime', 'req_from', 'req_to', 'amount', 
                                'registration_type_en'], keep='last', inplace=True)

    df.drop(['property_usage_id', 'transaction_subtype_id', 'transaction_type_id', 'property_id', 
            'transaction_subtype_en', 'is_freehold_text',
            'property_type_ar','property_type_id', 'property_subtype_ar',
            'property_subtype_id', 'building_age', 'rooms_ar', 'project_name_ar',
            'area_ar', 'area_id', 'nearest_landmark_ar', 'nearest_metro_ar', 'nearest_mall_ar',
            'master_project_ar', 'meta_ts', 'transaction_number',
            'req_from', 'transaction_datetime', 'req_to'
            ], axis=1, inplace=True)   

    numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

    # Replace NaN values in categorical features with 'missing'
    df = df.apply(lambda x: x.fillna('missing') if x.dtype == 'object' else x)
        
    categorical_features = df.select_dtypes(include=['object']).columns

    # Convert categorical features to numerical codes
    for col in categorical_features:
        df[col] = df[col].astype('category').cat.codes

    return df, numerical_features, categorical_features

In [9]:
df_preprocess_data_trans, numerical_features_trans, categorical_features_trans = preprocess_data_trans(trans)
df_preprocess_data_trans

,transaction_type_en,registration_type_en,property_usage_en,amount,total_buyer,total_seller,transaction_size_sqm,property_size_sqm,parcel_id,is_offplan,...,project_name_en,area_en,nearest_landmark_en,nearest_metro_en,nearest_mall_en,master_project_en,entry_id,delta_time_reg_to,delta_time_reg_from,time_reg
159419,1,1,1,2.228449e+09,0,0,7432243.24,7432243.24,NaN,False,...,2052,199,14,56,5,7,94367,24,24,-1
159658,1,1,0,5.520000e+08,0,0,10703.66,10703.66,NaN,False,...,2052,78,14,56,5,7,94367,23,24,0
162252,1,1,1,5.203507e+08,0,0,31384.41,31384.41,NaN,False,...,2052,119,14,56,0,7,94367,5,24,18
157251,1,1,1,5.030000e+08,0,0,12897.82,13747.52,6008142.0,False,...,2052,105,10,56,5,7,94367,4,24,19
159615,1,1,1,4.000000e+08,0,0,5416.09,5416.09,3450566.0,False,...,2052,101,3,18,1,7,94367,12,24,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8669,2,1,1,6.195700e+04,1,1,205.22,205.22,NaN,False,...,1807,220,1,14,4,7,94359,30,30,-1
14895,0,1,1,5.995989e+04,0,0,1.00,1136.76,1260815.0,False,...,2052,10,5,1,5,7,94359,9,30,20
8888,2,1,1,5.804500e+04,1,1,8.55,225.66,1271225.0,False,...,2052,146,5,2,0,7,94359,22,30,7
14684,0,1,1,3.017709e+04,0,0,12.34,136.53,3260865.0,False,...,503,153,5,7,0,7,94359,22,30,7


## Model creation

### catboost_model_rents

In [10]:
# Assuming 'contract_amount' is the target variable
target_column_rents = 'contract_amount'

In [11]:
def cross_validate_model(X, y, cat_features, cv=5):
    """
    Perform cross-validation on the CatBoost model and return the best model and evaluation metrics.
    
    :param X: DataFrame with features.
    :param y: Series with target variable.
    :param cat_features: List of categorical feature names.
    :param cv: Number of cross-validation folds.
    :return: Dictionary with RMSE, R2, MAE scores, best model.
    """
    model = CatBoostRegressor(loss_function='RMSE', verbose=0)
    param_grid = {
        'iterations': [100, 200],
        'depth': [4, 6, 8],
        'learning_rate': [0.01, 0.1]
    }
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', verbose=1)
    grid_search.fit(X, y, cat_features=cat_features)
    
    best_model = grid_search.best_estimator_
    
    # Cross-validation predictions
    y_pred = cross_val_score(best_model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    
    # Calculate metrics
    rmse_scores = np.sqrt(-y_pred)
    r2_scores = cross_val_score(best_model, X, y, cv=cv, scoring='r2')
    mae_scores = cross_val_score(best_model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    
    metrics = {
        'RMSE': (np.mean(rmse_scores), np.std(rmse_scores)),
        'R2': (np.mean(r2_scores), np.std(r2_scores)),
        'MAE': (np.mean(-mae_scores), np.std(-mae_scores))
    }
    
    return metrics, best_model

In [12]:
def train_and_evaluate_model(df, target_column, categorical_features):
    """
    Train and evaluate a CatBoost model.
    
    :param df: DataFrame with preprocessed data.
    :param target_column: Name of the target column.
    :param categorical_features: List of categorical feature names.
    :return: Trained model.
    """
    # Remove rows with NaN values in the target column
    df = df.dropna(subset=[target_column])
    
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Perform cross-validation and get the best model
    metrics, best_model = cross_validate_model(X, y, categorical_features)
    print(f"Cross-validated RMSE: {metrics['RMSE'][0]:.2f}")
    
    # Train the best model on the entire dataset
    best_model.fit(X, y, cat_features=categorical_features)
    
    return best_model, metrics


In [13]:
def save_model(model, filename):
    model.save_model(f'../models/{filename}', format="cbm")


In [ ]:

# Train and evaluate model
model_rents, metrics_rents = train_and_evaluate_model(df_preprocess_data_rents, target_column_rents, categorical_features_rents.tolist())

# Save the model
save_model(model_rents, 'catboost_model_rents.cbm')

metrics_rents

Fitting 5 folds for each of 12 candidates, totalling 60 fits


### catboost_model_trans

In [ ]:
# Assuming 'contract_amount' is the target variable
target_column_trans = 'amount'

# Train and evaluate model
model_trans, metrics_trans = train_and_evaluate_model(df_preprocess_data_trans, target_column_trans, categorical_features_trans.tolist())

# Save the model
save_model(model_trans, 'catboost_model_trans.cbm')

metrics_trans

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Cross-validated RMSE: 1267.65


{'RMSE': (1267.6486532050326, 93.37887768267211),
 'R2': (0.23553095235628332, 0.12454301117839774),
 'MAE': (1615652.7227698092, 228214.59209918356)}